In [ ]:
# Load and read PDF content
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader("pdf/FSI-Redevelopment.pdf")
docs = loader.load()
print("Number of pages in the PDF:", len(docs))

# Extract text from each page for embedding
doc_texts = [page.page_content for page in docs]
#print(" Document  texts",doc_texts)


Number of pages in the PDF: 32


In [ ]:
from milvus_model.hybrid import BGEM3EmbeddingFunction
import torch
device = "mps" if torch.backends.mps.is_available() else "cpu"
ef = BGEM3EmbeddingFunction(use_fp16=False, device=device)
dense_dim = ef.dim["dense"]

# Generate embeddings using BGE-M3 model
docs_embeddings = ef(doc_texts)

/Users/krishna/Documents/PROJECTS/pythonproject/Github/chatbotapp/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 189787.51it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
dense_dim

1024

In [ ]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [ ]:
# Connect to Milvus given URI
connections.connect(uri="./milvus.db")

# Specify the data schema for the new Collection
fields = [
    # Use auto generated id as primary key
    FieldSchema(
        name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100
    ),
    # Store the original text to retrieve based on semantically distance
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=2048),
    # Milvus now supports both sparse and dense vectors,
    # we can store each in a separate field to conduct hybrid search on both vectors
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),
]
schema = CollectionSchema(fields)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# Create collection (drop the old one if exists)
col_name = "hybrid_demo"
if utility.has_collection(col_name):
    Collection(col_name).drop()
col = Collection(col_name, schema, consistency_level="Strong")

# To make vector search efficient, we need to create indices for the vector fields
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
col.create_index("sparse_vector", sparse_index)
dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
col.create_index("dense_vector", dense_index)
col.load()

In [ ]:
# For efficiency, we insert 50 records in each small batch
for i in range(0, len(doc_texts), 50):
    # Extract the raw text from Document objects
    batched_texts = [doc.page_content for doc in docs[i : i + 50]]  # Extract the text content
    batched_sparse_vectors = docs_embeddings["sparse"][i : i + 50]
    batched_dense_vectors = docs_embeddings["dense"][i : i + 50]

    # Prepare the batched entities for insertion
    batched_entities = [
        batched_texts,          # Text content
        batched_sparse_vectors, # Sparse vector embeddings
        batched_dense_vectors,  # Dense vector embeddings
    ]

    # Insert the batch into Milvus
    col.insert(batched_entities)

print("Number of entities inserted:", col.num_entities)

Number of entities inserted: 32


In [ ]:
# Enter your search query
query = input("Enter your search query: ")
print(query)

# Generate embeddings for the query
query_embeddings = ef([query])
print(query_embeddings)

In [ ]:
from pymilvus import (
    AnnSearchRequest,
    WeightedRanker,
)

def dense_search(col, query_dense_embedding, limit=5):
    search_params = {"metric_type": "IP", "params": {}}
    res = col.search(
        [query_dense_embedding],
        anns_field="dense_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]

In [ ]:
def sparse_search(col, query_sparse_embedding, limit=5):
    search_params = {
        "metric_type": "IP", "params": {}, }
    res = col.search(
        [query_sparse_embedding],
        anns_field="sparse_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]

In [ ]:
def hybrid_search(
    col,
    query_dense_embedding,
    query_sparse_embedding,
    sparse_weight=1.0,
    dense_weight=1.0,
    limit=5,
):
    dense_search_params = {"metric_type": "IP", "params": {}}
    dense_req = AnnSearchRequest(
        [query_dense_embedding], "dense_vector", dense_search_params, limit=limit
    )
    sparse_search_params = {"metric_type": "IP", "params": {}}
    sparse_req = AnnSearchRequest(
        [query_sparse_embedding], "sparse_vector", sparse_search_params, limit=limit
    )
    rerank = WeightedRanker(sparse_weight, dense_weight)
    res = col.hybrid_search(
        [sparse_req, dense_req], rerank=rerank, limit=limit, output_fields=["text"]
    )[0]
    return [hit.get("text") for hit in res]

In [ ]:
dense_results = dense_search(col, query_embeddings["dense"][0])
sparse_results = sparse_search(col, query_embeddings["sparse"][[0]])
hybrid_results = hybrid_search(
    col,
    query_embeddings["dense"][0],
    query_embeddings["sparse"][[0]],
    sparse_weight=0.7,
    dense_weight=1.0,
)

In [ ]:
print(dense_results)

In [ ]:
print(sparse_results)

In [ ]:
print(hybrid_results)

In [ ]:
def doc_text_formatting(ef, query, docs):
    tokenizer = ef.model.tokenizer
    query_tokens_ids = tokenizer.encode(query, return_offsets_mapping=True)
    query_tokens = tokenizer.convert_ids_to_tokens(query_tokens_ids)
    formatted_texts = []

    for doc in docs:
        ldx = 0
        landmarks = []
        encoding = tokenizer.encode_plus(doc, return_offsets_mapping=True)
        tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])[1:-1]
        offsets = encoding["offset_mapping"][1:-1]
        for token, (start, end) in zip(tokens, offsets):
            if token in query_tokens:
                if len(landmarks) != 0 and start == landmarks[-1]:
                    landmarks[-1] = end
                else:
                    landmarks.append(start)
                    landmarks.append(end)
        close = False
        formatted_text = ""
        for i, c in enumerate(doc):
            if ldx == len(landmarks):
                pass
            elif i == landmarks[ldx]:
                if close:
                    formatted_text += "</span>"
                else:
                    formatted_text += "<span style='color:red'>"
                close = not close
                ldx = ldx + 1
            formatted_text += c
        if close is True:
            formatted_text += "</span>"
        formatted_texts.append(formatted_text)
    return formatted_texts

In [ ]:
from IPython.display import Markdown, display

# Dense search results
display(Markdown("**Dense Search Results:**"))
formatted_results = doc_text_formatting(ef, query, dense_results)
for result in dense_results:
    display(Markdown(result))

In [ ]:
# Sparse search results
display(Markdown("\n**Sparse Search Results:**"))
formatted_results = doc_text_formatting(ef, query, sparse_results)
for result in formatted_results:
    display(Markdown(result))

In [ ]:
# Hybrid search results
display(Markdown("\n**Hybrid Search Results:**"))
formatted_results = doc_text_formatting(ef, query, hybrid_results)
for result in formatted_results:
    display(Markdown(result))

In [ ]:
final_template = f"""
You are an expert assistant providing detailed and accurate information. The user has asked the following question:
**Question:** {query}

Below is relevant information retrieved from various sources:
**Retrieved Information:**
{hybrid_results}

**Instruction:** Use only the query-relevant content from the retrieved information to answer the question. Focus on providing a comprehensive, informative response based solely on the given data. If any conflicting details are present, prioritize the most reliable and consistent information.
"""
final_template

In [ ]:
from groq import Groq
client = Groq(
    api_key='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' ,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": final_template,
        }
    ],
    model="llama-3.1-70b-versatile",
)
print(chat_completion.choices[0].message.content)